In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

from sklearn.svm import SVC
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk.tokenize import TweetTokenizer

import matplotlib.pyplot as plt
'exec(%matplotlib inline)'
from subprocess import check_output

In [2]:
data = pd.read_csv(os.getcwd()+"/input/Sentiment_GOP.csv", encoding = "ISO-8859-1")
data = data[['text','sentiment']]

# Splitting the dataset into train and test set
train, test = train_test_split(data,test_size = 0.1)
# Removing neutral sentiments
train = train[train.sentiment != "Neutral"]
test = test[test.sentiment != "Neutral"]

train_pos = train[ train['sentiment'] == 'Positive']
train_pos = train_pos['text']
train_neg = train[ train['sentiment'] == 'Negative']
train_neg = train_neg['text']

In [3]:
tweets = []
stopwords_set = set(stopwords.words("english")) #stopwords are words like is, are, that
tknzr = TweetTokenizer()

for index, row in train.iterrows():
    words_filtered = [e.lower() for e in tknzr.tokenize(row.text) if len(e) >= 3] #gets rid of words that are shorter than 3
    words_cleaned = [word for word in words_filtered
        if 'http' not in word
        and not word.startswith('@')
        and not word.startswith('#')
        and word != 'RT']   #gets rid of words with http, @, #, or RT
    words_without_stopwords = [word for word in words_cleaned if not word in stopwords_set]  #gets rid of stopwords
    tweets.append((words_without_stopwords, row.sentiment))
    

test_pos = test[ test['sentiment'] == 'Positive']
test_pos = test_pos['text']
test_neg = test[ test['sentiment'] == 'Negative']
test_neg = test_neg['text']
test_set = test['text']
test_ans = test['sentiment']

In [4]:
# Extracting word features

#returns list of all words
def get_words_in_tweets(tweets):
    all = []
    for (words, sentiment) in tweets:
        all.extend(words)
    return all 

#
def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)   
    #FreqDist is a class that is like dictionary with word as key and # of times it appears as value

    features = wordlist.keys()
    return features

w_features = get_word_features(get_words_in_tweets(tweets)) #All unique words in all tweets

def extract_features(document):
    document_words = set(document)  #document was list of words, now list of unique words
    features = {}
    for word in w_features:
        features['contains(%s)' % word] = (word in document_words) #adds word from all unique words into features if that word is in document
    return features


In [5]:
training_set = nltk.classify.apply_features(extract_features,tweets)
classifier = nltk.classify.SklearnClassifier(SVC())
classifier.train(training_set)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


<SklearnClassifier(SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False))>

In [10]:
def accuracyScore(correct, total):
    if total == 0:
        return -1
    return correct/total      #How likely that u answer right

def sensitivityScore(truePos, totalPos):
    if totalPos == 0:
        return -1
    return truePos/totalPos    #How likely detect positive when is positive
    
def specificityScore(trueNeg, totalNeg):
    if totalNeg == 0:
        return -1
    return trueNeg/totalNeg   #How likely detect negative when is negative
    
def precisionScore(truePos, guessedPos):
    if guessedPos == 0:
        return -1
    return truePos/guessedPos #How likely is positive when detect positive

def f1Score(sensitivity, precision):
    if (sensitivity+precision) == 0:
        return -1
    return 2*sensitivity*precision/(sensitivity+precision)  
    #weighted avg of recall and precision, useful if harm of falsePos and falseNeg differs


In [7]:
# trueNeg = 0
# truePos = 0
# falseNeg = 0
# falsePos = 0

# #Detecting Negatives
# wrongNeg = []
# count = 0
# for obj in test_neg: 
#     processedObj = extract_features(nltk.word_tokenize(obj))
#     res =  classifier.classify(processedObj)
#     if(res == 'Negative'): 
#         trueNeg+=1
#     else:
#         falsePos+=1
#         if(count<20):
#             count+=1
#             wrongNeg.append(processedObj)
        
# #Detecting Positives
# wrongPos = []
# count = 0
# for obj in test_pos:
#     processedObj = extract_features(nltk.word_tokenize(obj))
#     res =  classifier.classify(processedObj)
#     if(res == 'Positive'): 
#         truePos+=1
#     else:
#         falseNeg+=1
#         if(count<20):
#             count+=1
#             wrongPos.append(processedObj)

In [8]:
trueNeg = 0
truePos = 0
falseNeg = 0
falsePos = 0

wrongNeg = []
wrongPos = []
for obj, ans in zip(test_set, test_ans): 
    res =  classifier.classify(extract_features(tknzr.tokenize(obj)))
    if(ans == 'Negative'):
        if(res == 'Negative'): 
            trueNeg+=1
        else:
            falsePos+=1
            if(len(wrongNeg)<10):
                wrongNeg.append([obj, ans])        
    else:
        if(res == 'Positive'): 
            truePos+=1
        else:
            falseNeg+=1
            if(len(wrongPos)<10):
                wrongPos.append([obj, ans])


In [11]:
print(str(truePos)+" "+ str(falseNeg))       
print(str(falsePos)+ " "+ str(trueNeg))  

accuracy = accuracyScore(truePos+trueNeg, truePos+trueNeg+falsePos+falseNeg)
sensitivity = sensitivityScore(truePos,truePos+falseNeg)
specificity = specificityScore(trueNeg, trueNeg+falsePos)
precision = precisionScore(truePos, truePos+falsePos)
f1 = f1Score(sensitivity, precision)

print(accuracy)
print(sensitivity)
print(specificity)
print(precision)
print(f1)

0 226
0 854
0.7907407407407407
0.0
1.0
-1
0.0


In [ ]:
for obj in wrongNeg:
    print(obj)
    print("\n")


In [ ]:
for obj in wrongPos:
    print(obj)
    print("\n")